<a href="https://colab.research.google.com/github/farazlfc/Quant-Learnings-and-Experiments/blob/version_2/pseduo_hangman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import random
from collections import defaultdict, Counter
import re
import string

In [37]:
dictionary_path = "/content/drive/MyDrive/trexquant/words_250000_train.txt"

In [38]:
words_dict = []
with open(dictionary_path, 'r') as file_:
  for x in file_:
    words_dict.append(x.strip())

In [39]:
print(words_dict[:1000])

['aaa', 'aaaaaa', 'aaas', 'aachen', 'aaee', 'aag', 'aahed', 'aahs', 'aal', 'aalesund', 'aaliis', 'aalst', 'aam', 'aandahl', 'aao', 'aapss', 'aar', 'aarau', 'aardvark', 'aardwolf', 'aaren', 'aargh', 'aarika', 'aaronic', 'aaronite', 'aaronsburg', 'aarp', 'aarrghh', 'aas', 'aasvogels', 'aaup', 'aavso', 'aba', 'ababdeh', 'abac', 'abacas', 'abacaxi', 'abaci', 'abacination', 'abaciscus', 'aback', 'abaco', 'abacterial', 'abactinally', 'abactor', 'abaculus', 'abacuses', 'abada', 'abaddon', 'abadengo', 'abadite', 'abaft', 'abagail', 'abailard', 'abaised', 'abaisse', 'abaka', 'abakas', 'abalation', 'abalienated', 'abalienation', 'abalones', 'abamp', 'abamperes', 'abana', 'abandon', 'abandoned', 'abandonee', 'abandoners', 'abandonment', 'abandons', 'abanet', 'abanic', 'abantes', 'abaptiston', 'abarambo', 'abaris', 'abarticular', 'abas', 'abased', 'abasedness', 'abasements', 'abasers', 'abasgi', 'abashed', 'abashedness', 'abashing', 'abashlessly', 'abashments', 'abasias', 'abasing', 'abask', 'abas

In [57]:
class Prediction():
  def __init__(self, words, tries_remains = 6):
    self.seen_letters = defaultdict(lambda:False)
    self.index = 0
    self.tries_remains = tries_remains
    self.game_status = False #final game status
    self.all_letters = list(string.ascii_lowercase)
    self.freq_tuples = []
    self.full_dictionary = words.copy()
    self.guessed = set()
    self.baseline_filtered_dict = words.copy()
    self.train_set = []
    self.test_set = []
    self.baseline_train_set = []
  
  def get_skeleton(self, word):
    return "_"*len(word)

  def clean_word(self, word):
      return word.replace("_",".")
  
  def split_train_test(self):
    random.shuffle(self.baseline_filtered_dict)
    self.train_set = self.baseline_filtered_dict[:-10000]
    self.test_set = self.baseline_filtered_dict[-10000:]
    self.baseline_train_set = self.train_set.copy()


  # add $ to the front of a word since this is a Bigram Model.
  def convert_word(self, word):
      return "$" + word

  # Collect bigram counts. Because we don't delete keys in dictionary and counter will return 0 for unseen pattern,
  # we don't need to add char for dictionary as Unigram Based on Word Length approach.
  def bigram(self, corpus):
      bigram_counts = defaultdict(Counter)
      
      for word in corpus:
          word = self.convert_word(word)
          
          # generate a list of bigrams
          bigram_list = zip(word[:-1], word[1:])
          
          # iterate over bigrams
          for bigram in bigram_list:
              first, second = bigram
              bigram_counts[first][second] += 1
      return bigram_counts
  
  def train_bigram(self):
      #self.bigram_counts = self.bigram(self.full_dictionary)
      self.bigram_counts = self.bigram(self.train_set)
      return
  
  def train_unigram(self):
    #self.unigram_counts = self.unigram(self.full_dictionary)
    self.unigram_counts = self.unigram(self.train_set)
    return

  # Calculate bigram probability
  def bigram_prob(self, key, char, bigram_counts):
    prev_word_counts = bigram_counts[key]
    total_counts = float(sum(prev_word_counts.values()))

    if total_counts == 0:
      return 0
    return prev_word_counts[char] / total_counts #all occurences with char next/ all occurences!



  def unigram(self, corpus):
      unigram_counts = Counter()
      
      for word in corpus:
          for char in set(word): #slight modification
              unigram_counts[char] += 1
              
      return unigram_counts

  def bigram_guess(self):
    available = list(set(string.ascii_lowercase) - self.guessed)
    print(available)
    # The probabilities of all available characters
    bigram_probs = [] 
    if "_" not in self.mask: #base condition
      return "SUCCESS"
    
    print(self.mask)

    for char in available:
        char_prob = 0
        for index in range(len(self.mask)):
            # The first case is that the first char has not been guessed
            if index == 0 and self.mask[index] == '_':
                char_prob +=  self.bigram_prob('$', char, self.bigram_counts)
                
            # The second case is that the other chars have not been guessed
            elif self.mask[index] == '_':
                # if the previous word has been guessed apply bigram
                if not self.mask[index - 1] == '_':
                    char_prob +=  self.bigram_prob(self.mask[index - 1], char, self.bigram_counts)
                    
                # If the previous word has not been guessed apply unigram
                else:
                    val = float(sum(self.unigram_counts.values()))
                    if val == 0:
                      char_prob  += 0
                    else:
                      char_prob +=  self.unigram_counts[char] / val
                
            # The final case is that the character is guessed so we skip this position
            else:
                continue
                
        bigram_probs.append(char_prob)
    
    # Return the max probability of char
    return available[bigram_probs.index(max(bigram_probs))]
                  

  def update_word(self, guess_letter):
    temp = ""
    if guess_letter in self.word:
      for index in range(len(self.word)):
        if self.word[index] == guess_letter:
          temp += self.word[index]
        else:
          temp += self.mask[index]
      return True, temp
    else:
      return False, self.word
    
  def train_grams(self):
      self.train_unigram()
      self.train_bigram()

  def init_2(self):
          #self.filter_dictionary() #call after assigning the word!
          self.train_unigram()
          self.train_bigram()

  def update_for_wrong_guess(self, letter):
    self.train_set = [x for x in self.train_set if letter not in x]


  
  def run_ngram(self, word, skeleton):
    self.word = word #to be accessible everywhere
    self.mask = skeleton
    self.init_2()
    tries_remains = self.tries_remains
    print(tries_remains)

    while tries_remains: 
        guess_letter = self.bigram_guess()

        if guess_letter == "SUCCESS":
          self.successes += 1
          self.game_status = True
          return
        self.guessed.add(guess_letter) #add to set of seen letters
        status, new_word = self.update_word(guess_letter)
        if status:
          self.mask = new_word #updated mask
          print(self.mask)
          #self.regex_filter()
          #self.train_grams()
        else:
          self.update_for_wrong_guess(guess_letter)
          self.train_grams()
          tries_remains -= 1

        if not tries_remains:
          self.fails += 1
          return
  
  def reset(self): #reset parameters
    self.successes, self.fails, self.game_status = 0, 0, False
    return

  def random_choose(self):
    #return random.choice(self.full_dictionary)
    return random.choice(self.test_set)
  
  def filter_dictionary(self):
    length = len(self.word)
    self.train_set = [x for x in self.train_set if len(x) == length]
    return
  
  def regex_filter(self):
    temp = self.mask.replace("_",".")
    self.train_set = [x for x in self.train_set if re.match(temp, x)]
    return


  def run_trials(self, n_simulations, train = True): #main function to run trials
    n_trials = 6 #hardcoded
    if train: #in case self.train_bigram had been called before
      self.split_train_test()
      self.train_unigram()
      self.train_bigram()
    
    
    self.reset() #reset parameters
    print("Running Trials now")
    for _ in range(n_simulations):
      self.train_set = self.baseline_train_set.copy()
      master_word = self.random_choose() #choose a random word from the test set
      master_word_skeleton = self.get_skeleton(master_word) #get its skeleton
      self.guessed = set() #as a part of init function!
      self.run_ngram(master_word, master_word_skeleton) #trigger bigram game with 6 tries

    print(self.successes)
    print(self.fails)

In [60]:
pred = Prediction(words_dict)
pred.run_trials(100)

Running Trials now
6
['h', 'p', 'v', 'x', 'd', 'a', 'f', 'z', 'm', 'r', 'q', 'b', 'j', 'k', 't', 'y', 'g', 'e', 'n', 'l', 'u', 'o', 's', 'i', 'c', 'w']
______
_e____
['h', 'p', 'v', 'x', 'd', 'a', 'f', 'z', 'm', 'r', 'q', 'b', 'j', 'k', 't', 'y', 'g', 'n', 'l', 'u', 'o', 's', 'i', 'c', 'w']
_e____
['h', 'p', 'v', 'x', 'd', 'a', 'f', 'z', 'm', 'q', 'b', 'j', 'k', 't', 'y', 'g', 'n', 'l', 'u', 'o', 's', 'i', 'c', 'w']
_e____
['h', 'p', 'v', 'x', 'd', 'a', 'f', 'z', 'm', 'q', 'b', 'j', 'k', 't', 'y', 'g', 'n', 'l', 'u', 'o', 'i', 'c', 'w']
_e____
_en___
['h', 'p', 'v', 'x', 'd', 'a', 'f', 'z', 'm', 'q', 'b', 'j', 'k', 't', 'y', 'g', 'l', 'u', 'o', 'i', 'c', 'w']
_en___
_en__a
['h', 'p', 'v', 'x', 'd', 'f', 'z', 'm', 'q', 'b', 'j', 'k', 't', 'y', 'g', 'l', 'u', 'o', 'i', 'c', 'w']
_en__a
_ent_a
['h', 'p', 'v', 'x', 'z', 'm', 'q', 'b', 'g', 'u', 'o', 'w', 'd', 'f', 'j', 'k', 'y', 'l', 'i', 'c']
_ent_a
['h', 'p', 'v', 'x', 'z', 'm', 'q', 'b', 'g', 'u', 'o', 'w', 'd', 'f', 'j', 'k', 'y', 'l',